In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time

from scraping_prices_cards import *

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def extract_html_code_1Y_NM_ppp(website, holofoil_price, reverse_holofoil_price):
    """
    Extracts price history data for Pokemon cards from TCGPlayer, handling both Holofoil and Reverse Holofoil variants.
    
    The function navigates through the TCGPlayer website interface to extract historical price data for a specific Pokemon card.
    It automatically selects between Holofoil and Reverse Holofoil based on provided prices, handling NaN values appropriately.
    
    Args:
        website (str): TCGPlayer URL for the specific Pokemon card
                      Format: "https://prices.pokemontcg.io/tcgplayer/{card_id}"
        holofoil_price (float): Current market price for Holofoil variant
                               Can be NaN if variant doesn't exist
        reverse_holofoil_price (float): Current market price for Reverse Holofoil variant
                                       Can be NaN if variant doesn't exist
    
    Returns:
        str or None: HTML content containing price history data if successful,
                    None if extraction fails or no valid prices found
    
    Example:
        >>> url = "https://prices.pokemontcg.io/tcgplayer/ex12-1"
        >>> holofoil = 21.21
        >>> reverse = 32.65
        >>> html_data = extract_html_code_1Y_NM_ppp(url, holofoil, reverse)
    
    Technical Process:
        1. Initializes WebDriver
        2. Navigates to card page
        3. Executes sequence of clicks:
           - Selects 1Y timeframe
           - Opens filters modal
           - Selects Near Mint condition
           - Chooses appropriate foil type
        4. Extracts price history data
    """
    driver = setup_driver()
    wait = WebDriverWait(driver, 40)
    
    try:
        # Load the webpage
        driver.get(website)
        print("Page loaded successfully")
        
        # Wait for page to be fully loaded
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
        # Define click sequence for data filtering
        initial_clicks = [
            ('CSS_SELECTOR', 'button[data-v-0177b97d][class*="charts-item"]:last-child'),   # 1Y timeframe button
            ('CSS_SELECTOR', 'div.modal__activator[role="button"]'),                        # View More Data button
            ('CSS_SELECTOR', 'button.sales-history-snapshot__show-filters'),                # Sales Filter button
            ('XPATH', '//label[span[text()="Near Mint"]]')                                  # Near Mint condition filter
        ]
        
        # Execute click sequence
        for selector_type, selector in initial_clicks:
            by_type = By.CSS_SELECTOR if selector_type == 'CSS_SELECTOR' else By.XPATH
            if not test_button_click(driver, wait, selector, by=by_type):
                print(f"Click failed for selector: {selector}")
                return None
        
        # If the price is a Nan value
        holofoil_valid = not pd.isna(holofoil_price)
        reverse_valid = not pd.isna(reverse_holofoil_price)
        
        if holofoil_valid and reverse_valid:
            # we take the highest price ! 
            if holofoil_price > reverse_holofoil_price:
                test_button_click(driver, wait, 
                    '//label[contains(., "Holofoil")]', By.XPATH)
            else:
                test_button_click(driver, wait, 
                    '//label[contains(., "Reverse Holofoil")]', By.XPATH)
        elif holofoil_valid:
            # If the holofoil price is valid
            test_button_click(driver, wait, 
                '//label[contains(., "Holofoil")]', By.XPATH)
        elif reverse_valid:
            # If the valid holofoil price is valid
            test_button_click(driver, wait, 
                '//label[contains(., "Reverse Holofoil")]', By.XPATH)
        else:
            # Aucun prix valide
            print("No valid prices found")
            return None
            
        # Extract chart data
        html_content = get_chart_data(driver, wait)
        if html_content:
            print("HTML content successfully retrieved")
            return html_content
            
        print("Failed to retrieve HTML content")
        return None
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
        
    finally:
        driver.quit()
        print("Browser session terminated")

In [23]:
def extract_price_history(html_content):
    """
    Extracts and processes price history data from a Pokemon card website.
    
    This function parses HTML content containing price history data for Pokemon cards,
    processes date ranges, prices, and quantities, and returns a structured DataFrame.
    
    Args:
        html_content (str): Raw HTML content containing price history table data
        
    Returns:
        pandas.DataFrame: A sorted DataFrame containing:
            - MultiIndex: (start_date, end_date) as datetime objects
            - Columns: 
                - price (float): Card price for the period
                - quantity_sold (int): Number of cards sold
                
    Example:
        >>> html_data = get_html_content()
        >>> price_df = extract_price_history_(html_data)
    """
    soup = BeautifulSoup(html_content, "html.parser")
    price_history = {}
    
    def convert_date(date_str):
        """
        Converts date range strings to datetime objects with proper year assignment.
        
        Handles date ranges that span across years (Dec-Jan) by automatically
        assigning the correct year to each date.
        
        Args:
            date_str (str): Date range string in format "MM/DD to MM/DD"
            
        Returns:
            tuple: (start_date, end_date) as datetime objects
                  Returns (None, None) if conversion fails
        """
        try:
            start_date, end_date = date_str.split(" to ")
            start_month, start_day = map(int, start_date.split("/"))
            end_month, end_day = map(int, end_date.split("/"))
            
            current_year = 2024
            previous_year = current_year - 1
            
            start_year = previous_year if start_month == 12 else current_year
            end_year = current_year if end_month == 1 else start_year
            
            return datetime(start_year, start_month, start_day), datetime(end_year, end_month, end_day)
            
        except Exception as e:
            print(f"Date conversion error: {date_str} - {str(e)}")
            return None, None

    rows = soup.find_all("tr")
    
    for row in rows[1:]:
        cells = row.find_all("td")
        if len(cells) >= 3:
            try:
                date = cells[0].get_text(strip=True)
                price = float(cells[1].get_text(strip=True).replace('$', ''))
                quantity = float(cells[2].get_text(strip=True).replace('$', ''))
                
                start_date_obj, end_date_obj = convert_date(date)
                if start_date_obj and end_date_obj:
                    price_history[(start_date_obj, end_date_obj)] = {
                        'price': price,
                        'quantity_sold': int(quantity)
                    }
            except (ValueError, AttributeError) as e:
                continue

    df = pd.DataFrame.from_dict(price_history, orient='index')
    df.index = pd.MultiIndex.from_tuples(df.index, names=['start_date', 'end_date'])
    return df.sort_index()

In [68]:
# Exemple d'utilisation
website = "https://prices.pokemontcg.io/tcgplayer/ex12-1"
holofoil_price = 51.21
reverse_holofoil_price = 32.65
html_code = extract_html_code_1Y_NM_ppp(website, holofoil_price, reverse_holofoil_price)

Page loaded successfully
HTML content successfully retrieved
Browser session terminated


In [69]:
html_code

'<div data-v-de762a76="" class="martech-charts-history" data-testid="History"><div data-v-de762a76="" class="martech-charts-content"><!----><div data-v-de762a76="" class="martech-charts-header-controls"><div data-v-eefc313a="" data-v-de762a76="" class="charts-header" data-testid="History__Default"><div data-v-eefc313a="" class="charts-row"><div data-v-eefc313a="" class="charts-color" style="background-color: rgb(242, 249, 255);"><div data-v-eefc313a="" class="charts-group" style="color: rgb(8, 53, 219);"><svg data-v-eefc313a="" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 40 28" class="graph-icon"><path d="M20.5 28c-1.1 0-2-.9-2-2V3h-7v10.5c0 1.1-.9 2-2 2h-9c-.28 0-.5-.22-.5-.5v-2c0-.28.22-.5.5-.5h8V2c0-1.1.9-2 2-2h9c1.1 0 2 .9 2 2v23h7V14.5c0-1.1.9-2 2-2h9c.28 0 .5.22.5.5v2c0 .28-.22.5-.5.5h-8V26c0 1.1-.9 2-2 2h-9Z" fill="currentColor"></path></svg><div data-v-eefc313a="" class="charts-title">Near Mint Holofoil </div></div><div data-v-eefc313a="" class="charts-price">$21.21</div></

In [70]:
price=extract_price_history_(html_code)

In [71]:
price

,,price,quantity_sold
start_date,end_date,,
2023-12-05,2023-12-11,16.73,0
2023-12-12,2023-12-18,16.73,0
2023-12-19,2023-12-25,16.73,0
2023-12-26,2024-01-01,16.67,1
2024-01-02,2024-01-08,16.46,0
2024-01-09,2024-01-15,16.46,0
2024-01-16,2024-01-22,16.46,0
2024-01-23,2024-01-29,16.60,1
2024-01-30,2024-02-05,16.60,0


In [4]:
from plot_card_prices_history import plot_price_quantity_variation_seaborn

In [6]:
plot_price_quantity_variation_seaborn(price)

NameError: name 'price' is not defined

In [15]:
ts_cards=pd.read_csv('pokemon_cards_15.csv')
test=ts_cards.sample(4)

In [16]:
test

,id,name,rarity,collection,series,release_date,nationalPokedexNumbers,artist,holofoil_price,reverse_holofoil_price
288,ecard2-H14,Kingdra,Rare Holo,Aquapolis,E-Card,2003/01/15,[230],Atsuko Nishida,92.25,NaN
1035,sm7-163,Copycat,Rare Ultra,Celestial Storm,Sun & Moon,2018/08/03,NaN,Megumi Mizutani,22.78,NaN
918,bw9-64,Umbreon,Rare Holo,Plasma Freeze,Black & White,2013/05/08,[197],5ban Graphics,27.71,37.23
1259,sm6-145,Mysterious Treasure,Rare Secret,Forbidden Light,Sun & Moon,2018/05/04,NaN,Eske Yoshinob,17.36,NaN


In [17]:
import os
import pandas as pd
import time
from tqdm import tqdm

def sequential_price_extraction(cards_df, output_dir='price_history'):
    os.makedirs(output_dir, exist_ok=True)
    
    progress_bar = tqdm(total=len(cards_df), desc="Extraction des prix")
    
    for index, card_row in cards_df.iterrows():
        card_id = card_row['id']
        price_holofoil=card_row["holofoil_price"]
        price_reverse_holofoil=card_row["reverse_holofoil_price"]
        file_path = os.path.join(output_dir, f'{card_id}.csv')
        
        if os.path.exists(file_path):
            progress_bar.set_description(f"Fichier existant pour {card_id}")
            progress_bar.update(1)
            continue
            
        try:
            html_code=extract_html_code_1Y_NM_ppp(f"https://prices.pokemontcg.io/tcgplayer/{card_id}",price_holofoil,price_reverse_holofoil)
            price_history = extract_price_history_(html_code)
            
            # Réinitialisation de l'index pour avoir les dates en colonnes
            price_df = price_history.reset_index()
            
            # Renommage des colonnes pour correspondre au format souhaité
            price_df.columns = ['start_date', 'end_date', 'price', 'quantity_sold']
            
            # Sauvegarde au format CSV
            price_df.to_csv(file_path, index=False)
            
            progress_bar.set_description(f"Extraction réussie pour {card_id}")
            
        except Exception as e:
            progress_bar.set_description(f"Échec pour {card_id}: {str(e)}")
        
        progress_bar.update(1)
        time.sleep(1)
    
    progress_bar.close()

In [22]:
sequential_price_extraction(test)

Fichier existant pour ecard2-H14:   0%|          | 0/4 [00:00<?, ?it/s]

Page loaded successfully


Extraction réussie pour sm7-163:  50%|█████     | 2/4 [01:18<01:18, 39.25s/it]

HTML content successfully retrieved
Browser session terminated


Fichier existant pour sm6-145: 100%|██████████| 4/4 [01:19<00:00, 19.88s/it]  


In [1]:
from scraping_prices_cards import sequential_price_extraction

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
import pandas as pd
ts_cards=pd.read_csv('pokemon_cards.csv')
test=ts_cards.sample(5)
test

,id,name,rarity,collection,series,release_date,nationalPokedexNumbers,artist,holofoil_price,reverse_holofoil_price
332,ex6-8,Nidoking,Rare Holo,FireRed & LeafGreen,EX,2004/09/01,[34],Hisao Nakamura,20.84,34.95
997,ex4-37,Team Magma's Mightyena,Uncommon,Team Magma vs Team Aqua,EX,2004/03/01,[262],Shin-ichi Yoshikawa,NaN,7.65
3720,sv8-234,Lisia's Appeal,Ultra Rare,Surging Sparks,Scarlet & Violet,2024/11/08,NaN,NaN,7.11,NaN
922,ex7-32,Dark Dragonair,Uncommon,Team Rocket Returns,EX,2004/11/01,[148],Tomoaki Imakuni,NaN,29.50
337,base6-12,Gyarados,Rare Holo,Legendary Collection,Other,2002/05/24,[130],Mitsuhiro Arita,45.00,148.85


In [8]:
test.tail(2)

,id,name,rarity,collection,series,release_date,nationalPokedexNumbers,artist,holofoil_price,reverse_holofoil_price
1391,ex15-3,Heracross δ,Rare Holo,Dragon Frontiers,EX,2006/11/01,[214],Mitsuhiro Arita,15.02,11.01
191,swsh10-172,Machamp V,Rare Ultra,Astral Radiance,Sword & Shield,2022/05/27,[68],Shinya Komatsu,124.25,NaN


In [4]:
sequential_price_extraction(test)

Price Extraction:   0%|          | 0/5 [00:00<?, ?it/s]

Selecting Reverse Holofoil


Extraction successful for ex6-8:  20%|██        | 1/5 [01:41<06:47, 101.90s/it]

Browser session terminated


Extraction successful for ex4-37:  40%|████      | 2/5 [03:18<04:56, 98.72s/it] 

...

Extraction successful for ex7-32:  80%|████████  | 4/5 [06:42<01:38, 98.70s/it]  

Browser session terminated
Selecting Reverse Holofoil


Extraction successful for base6-12: 100%|██████████| 5/5 [08:06<00:00, 93.53s/it]

Browser session terminated


Extraction successful for base6-12: 100%|██████████| 5/5 [08:07<00:00, 97.57s/it]
